In [2]:
!python --version

Python 3.11.0


In [3]:
from pyqubo import *
from neal import SimulatedAnnealingSampler
import time
# from dwave.system import LeapHybridSampler
import dimod
from tqdm import tqdm
import random
from multiprocessing import Pool
import networkx as nx

import os
from fnmatch import fnmatch
import time
import pandas as pd

In [4]:
def load_graph(path, prob):
    g = nx.DiGraph()
    s = None
    t = None
    with open(path) as f:
        for line in f:
            if line.startswith('p'):
                num_nodes = int(line.split(' ')[2])
                nodes = {x + 1 for x in range(num_nodes)}
                for n in nodes:
                    g.add_node(n)
            elif line.startswith('a'):
                vals = [int(x) for x in line.split(' ')[1:]]
                if (vals[2] > 0) and ((vals[0] == s or vals[1] == t) or (random.uniform(0, 1) >= prob)):
                    g.add_edge(vals[0], vals[1], capacity=vals[2])
            elif line.startswith('n'):
                line = line.strip().split(' ')
                if line[2] == 's':
                    s = int(line[1])
                elif line[2] == 't':
                    t = int(line[1])
    return g, s, t

In [5]:
def get_max_flow_from_cut_edges(graph, sample, var_name):
    cut_edges = {key: value for key, value in sample[0].items() if key.startswith(var_name) and value == 1}
    edges = set()
    for e in cut_edges:
        parts = e.split('_')
        edges.add((int(parts[1]), int(parts[2])))
    return sum(graph[i][j]['capacity'] for i, j in edges)

In [6]:
paths = []
import os

os.getcwd()
for path, subdirs, files in os.walk('data/'):
    paths.extend(os.path.join(path, name) for name in files if fnmatch(name, '*.max'))

graph_path = paths[10]

print(graph_path)
graph, source, target = load_graph(graph_path, 0.99999999999)
print(len(graph.edges()))
start_time = time.time()
lagrange = Placeholder('L')
# obj = 0
# omega_vars = {(i, j): Binary(f'omega_{i}_{j}') for i, j in graph.edges}
# pi_vars = {n: Binary(f'pi_{n}') for n in graph.nodes}
obj = Constraint((Binary(f'pi_{target}') - Binary(f'pi_{source}') - 1) ** 2,
                label='constraint_1', condition=lambda x: x == 0) * lagrange  # First constraint


data/BVZ-tsukuba8.max
62914


In [7]:
from pyqubo import Binary
a, b = Binary("a"), Binary("b")
a + b


(Binary('a') + Binary('b'))

In [ ]:
from tqdm import tqdm

# for y_count, (i, j, cap) in tqdm(enumerate(graph.edges(data=True))):
#     obj = obj + (Binary(f'omega_{i}_{j}') * cap['capacity']) # Obj function
#     obj = obj + Constraint(
#         (Binary(f'pi_{i}') - Binary(f'pi_{j}') + Binary(f'omega_{i}_{j}') -
#           (Binary(f'y_2^0-{y_count}') + 2 * Binary(f'y_2^1-{y_count}'))) ** 2,
#         label='constraint_2', condition=lambda x: x == 0
#     ) * lagrange  # Second constraint

print("ciccino")

bqm = obj.compile().to_bqm(feed_dict={'L': 15})
time_before_starting_annealer = time.time() - start_time

'''sampler = LeapHybridSampler(solver={'category': 'hybrid'}, token='DEV-6dda84212a7a7699b2236b18c96b2561c83b29b1')
res = sampler.sample(bqm, time_limit=56)

q_cut = res.first
q_mxflow = get_max_flow_from_cut_edges(graph, q_cut, 'omega')
q_time = time.time() - start_time
print(f'{time_before_starting_annealer=}')
print(f'{q_time=}')
print(f'{q_mxflow=}')
print('qpu_access_time', res.info['qpu_access_time'])'''

62914it [00:03, 20866.90it/s]

In [ ]:
import numpy as np
xx = [1, 100, 1000]
yy = [0.1, 10., 20.]
np.interp([50], xx, yy)[0]

In [ ]:
paths = []
import os

os.getcwd()
for path, subdirs, files in os.walk('data/'):
    paths.extend(os.path.join(path, name) for name in files if fnmatch(name, '*.max'))

graph_path = paths[10]

print(graph_path)
graph, source, target = load_graph(graph_path, 0.99)
print(len(graph.edges()))
start_time = time.time()
# lagrange = Placeholder('L')
# obj = 0
# omega_vars = {(i, j): Binary(f'omega_{i}_{j}') for i, j in graph.edges}
# pi_vars = {n: Binary(f'pi_{n}') for n in graph.nodes}
# obj = Constraint((Binary(f'pi_{target}') - Binary(f'pi_{source}') - 1) ** 2,
#                 label='constraint_1', condition=lambda x: x == 0) * lagrange  # First constraint
from dimod import BinaryQuadraticModel, Binary, Integer
import dwavebinarycsp

# fill with obj
linear = {f'omega_{i}_{j}' : cap["capacity"] for i, j ,cap in graph.edges(data=True)}

# fill with second constraints
quadratic = {}
for y_count, (i, j, cap) in tqdm(enumerate(graph.edges(data=True))):
    quadratic[f'pi_{i}**2'] = 1
    quadratic[f'pi_{j}**2'] = 1
    quadratic[f'omega{i}_{j}**2'] = 1
    quadratic[f'y_0_{y_count}**2'] = 1
    quadratic[f'y_1_{y_count}**2'] = 4
    quadratic[f'pi{i}*pi_{j}'] = -2
    quadratic[f'pi_{i}*omega_{i}_{j}'] = 2
    quadratic[f'pi_{i}*y_0_{y_count}'] = -2
    quadratic[f'pi_{i}*y_1_{y_count}'] = -4
    quadratic[f'pi_{j}*omega_{i}_{j}'] = -2
    quadratic[f'pi_{j}*y_0_{y_count}'] = 2
    quadratic[f'pi_{j}*y_1_{y_count}'] = 4
    quadratic[f'omega_{i}_{j}*y_0_{y_count}'] = -2
    quadratic[f'omega_{i}_{j}*y_1_{y_count}'] = -4
    quadratic[f'y_0_{y_count}*y_1_{y_count}'] = 4

# fill with first constraint
quadratic[f'pi_{target}**2'] = 1
quadratic[f'pi_{source}**2'] = 1
quadratic[f'pi_{target}_{source}'] = 2
linear[f'pi_{target}'] = -2
linear[f'pi_{source}'] = 2

bqm = BinaryQuadraticModel(linear, quadratic, -1, Binary)